In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import warnings
warnings.filterwarnings('ignore')

In [25]:
#Check the general info
bom_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [26]:
#loading data
bom_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
bom_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [27]:


# loading data for movie information
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t')
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [28]:
# loading data for ratings and reviews
ratings= pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter='\t', encoding='latin-1')
ratings.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [29]:
#the tmdb_dataset
tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz', index_col=0)
tmdb_movies.shape

(26517, 9)

In [30]:
# loading data for movie budget
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [31]:
#loading the data from the database
conn=sqlite3.connect("zippedData/im.db/im.db")
cur= conn.cursor()


In [32]:
query = """ 
SELECT 
    mb.movie_id,
    mb.primary_title,
    mb.original_title,
    mb.genres,
    mb.start_year AS release_year,
    mb.runtime_minutes AS film_runtime,
    mr.averagerating AS imdb_average_rating,
    mr.numvotes AS imdb_number_of_votes,
    GROUP_CONCAT(DISTINCT(p.primary_name)) AS directors
FROM movie_basics AS mb
LEFT JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
JOIN
directors AS d
USING (movie_id)
JOIN
persons AS p
USING (person_id)
GROUP BY mb.movie_id;
"""

imdb_df=pd.read_sql(query,conn)

In [33]:
imdb_df.head()

,movie_id,primary_title,original_title,genres,release_year,film_runtime,imdb_average_rating,imdb_number_of_votes,directors
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",2013,175.0,7.0,77.0,Harnam Singh Rawail
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",2019,114.0,7.2,43.0,Mani Kaul
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama,2018,122.0,6.9,4517.0,Orson Welles
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",2018,NaN,6.1,13.0,Hrishikesh Mukherjee
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",2017,80.0,6.5,119.0,"Raoul Ruiz,Valeria Sarmiento"


### DATA WRANGLING

In [34]:
# Exploring the TMDB movies dataset
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


The tmdb_movies has no missing values

## Exploring the goss income 

In [35]:
bom_gross.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [36]:
# Filling missing values with the mode in 'domestic_gross' column
domestic_mode = bom_gross['domestic_gross'].mode().values[0]
bom_gross['domestic_gross'].fillna(domestic_mode, inplace=True)
#filling missing values in studio with unknown
bom_gross['studio'].fillna('unknown')

# Filling missing values with the mode in 'foreign_gross' column
foreign_mode = bom_gross['foreign_gross'].mode().values[0]
bom_gross['foreign_gross'].fillna(foreign_mode, inplace=True)

# Converting domestic_gross column to string
bom_gross['domestic_gross'] = bom_gross['domestic_gross'].astype(str)

# Converting foreign_gross column to string
bom_gross['foreign_gross'] = bom_gross['foreign_gross'].astype(str)

# Remove commas from domestic_gross column values and convert to float
bom_gross['domestic_gross'] = bom_gross['domestic_gross'].str.replace(',', '').astype(float)

# Remove commas from foreign_gross column values and convert to float
bom_gross['foreign_gross'] = bom_gross['foreign_gross'].str.replace(',', '').astype(float)
bom_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3387 non-null   float64
 3   foreign_gross   3387 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


### All our columns in the gross income  are now populated

## Exploring the movie budget


In [37]:
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [38]:
movie_budget = movie_budget.drop('release_date', axis=1)

In [39]:
movie_budget['production_budget'] = movie_budget['production_budget'].str.replace('[\$,]', '', regex=True).astype(float)
movie_budget['domestic_gross'] = movie_budget['domestic_gross'].str.replace('[\$,]', '', regex=True).astype(float)
movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].str.replace('[\$,]', '', regex=True).astype(float)


In [40]:
movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].apply(lambda x: float(x.replace(',', '').replace('$', '')) if isinstance(x, str) else x)


In [41]:
movie_budget['movie']

0                                            Avatar
1       Pirates of the Caribbean: On Stranger Tides
2                                      Dark Phoenix
3                           Avengers: Age of Ultron
4                 Star Wars Ep. VIII: The Last Jedi
                           ...                     
5777                                         Red 11
5778                                      Following
5779                  Return to the Land of Wonders
5780                           A Plague So Pleasant
5781                              My Date With Drew
Name: movie, Length: 5782, dtype: object

In [42]:
movie_budget.describe()

,id,production_budget,domestic_gross,worldwide_gross
count,5782.000000,5.782000e+03,5.782000e+03,5.782000e+03
mean,50.372363,3.158776e+07,4.187333e+07,9.148746e+07
std,28.821076,4.181208e+07,6.824060e+07,1.747200e+08
min,1.000000,1.100000e+03,0.000000e+00,0.000000e+00
25%,25.000000,5.000000e+06,1.429534e+06,4.125415e+06
50%,50.000000,1.700000e+07,1.722594e+07,2.798445e+07
75%,75.000000,4.000000e+07,5.234866e+07,9.764584e+07
max,100.000000,4.250000e+08,9.366622e+08,2.776345e+09


# Looking into Movie info

In [43]:
# selecting useful columns
new_movie_info = movie_info[['id', 'genre', 'director', 'writer']]
new_movie_info.head()

,id,genre,director,writer
0,1,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman
1,3,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo
2,5,Drama|Musical and Performing Arts,Allison Anders,Allison Anders
3,6,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton
4,7,Drama|Romance,Rodney Bennett,Giles Cooper


In [44]:
# displaying the information of the data set
new_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1560 non-null   int64 
 1   genre     1552 non-null   object
 2   director  1361 non-null   object
 3   writer    1111 non-null   object
dtypes: int64(1), object(3)
memory usage: 48.9+ KB


In [45]:
new_movie_info.describe()

,id
count,1560.000000
mean,1007.303846
std,579.164527
min,1.000000
25%,504.750000
50%,1007.500000
75%,1503.250000
max,2000.000000


In [46]:
# Exploring the columns
new_movie_info['genre'].isna().sum()

8

In [47]:
new_movie_info['genre']


0                     Action and Adventure|Classics|Drama
1                       Drama|Science Fiction and Fantasy
2                       Drama|Musical and Performing Arts
3                              Drama|Mystery and Suspense
4                                           Drama|Romance
                              ...                        
1555     Action and Adventure|Horror|Mystery and Suspense
1556                   Comedy|Science Fiction and Fantasy
1557    Classics|Comedy|Drama|Musical and Performing Arts
1558      Comedy|Drama|Kids and Family|Sports and Fitness
1559    Action and Adventure|Art House and Internation...
Name: genre, Length: 1560, dtype: object

In [48]:
# Replacing the empty columns with uknown
new_movie_info['genre'] = new_movie_info['genre'].fillna('unkown')
new_movie_info['genre'].isna().sum()

0

In [49]:
new_movie_info['director']

0         William Friedkin
1         David Cronenberg
2           Allison Anders
3           Barry Levinson
4           Rodney Bennett
               ...        
1555                   NaN
1556          Steve Barron
1557        Gordon Douglas
1558    David Mickey Evans
1559                   NaN
Name: director, Length: 1560, dtype: object

In [50]:
new_movie_info['director'] = new_movie_info['director'].fillna('uknown')
new_movie_info['director'].isna().sum()

0

In [51]:
new_movie_info['writer']


0                                         Ernest Tidyman
1                           David Cronenberg|Don DeLillo
2                                         Allison Anders
3                        Paul Attanasio|Michael Crichton
4                                           Giles Cooper
                              ...                       
1555                                                 NaN
1556    Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner
1557                                                 NaN
1558                    David Mickey Evans|Robert Gunter
1559                                          Luc Besson
Name: writer, Length: 1560, dtype: object

In [52]:
new_movie_info['writer'] = new_movie_info['writer'].fillna('unkown')
new_movie_info['writer'].isna().sum()

0

In [53]:
new_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1560 non-null   int64 
 1   genre     1560 non-null   object
 2   director  1560 non-null   object
 3   writer    1560 non-null   object
dtypes: int64(1), object(3)
memory usage: 48.9+ KB


# Exploring the movie review

In [54]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [55]:
ratings['rating'].isna().sum()


13517

In [56]:
ratings['rating'] = ratings['rating'].fillna('1/1')


In [57]:
ratings = ratings[['id','rating','top_critic','date']]

In [58]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   rating      54432 non-null  object
 2   top_critic  54432 non-null  int64 
 3   date        54432 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.7+ MB


# Exploring the imdb df

In [59]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140416 entries, 0 to 140415
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   movie_id              140416 non-null  object 
 1   primary_title         140416 non-null  object 
 2   original_title        140414 non-null  object 
 3   genres                137238 non-null  object 
 4   release_year          140416 non-null  int64  
 5   film_runtime          112019 non-null  float64
 6   imdb_average_rating   73103 non-null   float64
 7   imdb_number_of_votes  73103 non-null   float64
 8   directors             140416 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 9.6+ MB


In [60]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140416 entries, 0 to 140415
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   movie_id              140416 non-null  object 
 1   primary_title         140416 non-null  object 
 2   original_title        140414 non-null  object 
 3   genres                137238 non-null  object 
 4   release_year          140416 non-null  int64  
 5   film_runtime          112019 non-null  float64
 6   imdb_average_rating   73103 non-null   float64
 7   imdb_number_of_votes  73103 non-null   float64
 8   directors             140416 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 9.6+ MB


In [61]:
imdb_new = imdb_df[['movie_id','primary_title','genres','directors']]

In [62]:
imdb_new.isna().sum()

movie_id            0
primary_title       0
genres           3178
directors           0
dtype: int64

In [63]:
#we replace the null values in the genres with unknown
imdb_new['genres']=imdb_df['genres'].fillna('unknown')

In [64]:
imdb_new.isna().sum()

movie_id         0
primary_title    0
genres           0
directors        0
dtype: int64

In [65]:
imdb_new.dropna()

,movie_id,primary_title,genres,directors
0,tt0063540,Sunghursh,"Action,Crime,Drama",Harnam Singh Rawail
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",Mani Kaul
2,tt0069049,The Other Side of the Wind,Drama,Orson Welles
3,tt0069204,Sabse Bada Sukh,"Comedy,Drama",Hrishikesh Mukherjee
4,tt0100275,The Wandering Soap Opera,"Comedy,Drama,Fantasy","Raoul Ruiz,Valeria Sarmiento"
...,...,...,...,...
140411,tt9916538,Kuambil Lagi Hatiku,Drama,Azhar Kinoi Lubis
140412,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Documentary,"Angela Gurgel,Ana Célia de Oliveira"
140413,tt9916706,Dankyavar Danka,Comedy,Kanchan Nayak
140414,tt9916730,6 Gunn,unknown,Kiran Gawade


In [66]:
imdb_new=imdb_new.rename(columns={'directors': 'director'})


In [67]:
imdb_new = imdb_new.rename(columns={'genres': 'genre'})

In [68]:
imdb_new = imdb_new.rename(columns={'primary_title': 'title'})

# Merging the datasets

In [69]:
merged1 = pd.merge(imdb_new,tmdb_movies,on ='title' ,how = 'inner')

In [70]:
merged1

,movie_id,title,genre,director,genre_ids,id,original_language,original_title,popularity,release_date,vote_average,vote_count
0,tt0069049,The Other Side of the Wind,Drama,Orson Welles,"[35, 18]",299782,en,The Other Side of the Wind,9.800,2018-11-02,7.0,64
1,tt0112502,Bigfoot,"Horror,Thriller",Mc Jones,"[27, 878, 12]",117856,en,Bigfoot,2.813,2012-06-30,2.9,26
2,tt4503112,Bigfoot,"Action,Horror",David Heavener,"[27, 878, 12]",117856,en,Bigfoot,2.813,2012-06-30,2.9,26
3,tt9181914,Bigfoot,"Animation,Family",Evan Tramel,"[27, 878, 12]",117856,en,Bigfoot,2.813,2012-06-30,2.9,26
4,tt0192528,Heaven & Hell,Drama,Stuart Paul,[18],567662,en,Heaven & Hell,0.600,2018-11-06,7.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...
23201,tt9678886,Vacancy,Documentary,Alexandra Longuet,"[35, 27]",475066,en,Vacancy,0.649,2017-10-27,9.0,1
23202,tt9777830,John Leguizamo's Latin History for Morons,Comedy,Aram Rappaport,[35],557606,en,John Leguizamo's Latin History for Morons,4.304,2018-11-05,7.3,13
23203,tt9814730,The Flare,Sport,Federico Vitetta,[],469698,en,The Flare,0.600,2017-07-11,9.0,1
23204,tt9862978,Terra,Documentary,"Hiroatsu Suzuki,Rossana Torres",[27],536235,en,Terra,1.767,2018-04-24,5.0,1


In [71]:
merged_2 = pd.merge(merged1,bom_gross,on = 'title',how = 'inner')
merged_2

,movie_id,title,genre,director,genre_ids,id,original_language,original_title,popularity,release_date,vote_average,vote_count,studio,domestic_gross,foreign_gross,year
0,tt0315642,Wazir,"Action,Crime,Drama",Bejoy Nambiar,"[53, 28, 80, 18, 9648]",275269,hi,वज़ीर,3.881,2016-01-08,6.6,63,Relbig.,1100000.0,1200000.0,2016
1,tt0337692,On the Road,"Adventure,Drama,Romance",Walter Salles,"[12, 18]",83770,en,On the Road,8.919,2012-12-21,5.6,518,IFC,744000.0,8000000.0,2012
2,tt2404548,On the Road,Drama,Harutyun Azaryan,"[12, 18]",83770,en,On the Road,8.919,2012-12-21,5.6,518,IFC,744000.0,8000000.0,2012
3,tt3872966,On the Road,Documentary,Marvin Dau,"[12, 18]",83770,en,On the Road,8.919,2012-12-21,5.6,518,IFC,744000.0,8000000.0,2012
4,tt4339118,On the Road,Drama,Poong-ki Kim,"[12, 18]",83770,en,On the Road,8.919,2012-12-21,5.6,518,IFC,744000.0,8000000.0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704,tt8267604,Capernaum,Drama,Nadine Labaki,[18],517814,ar,کفرناحوم,20.249,2018-12-14,8.4,329,SPC,1700000.0,1200000.0,2018
3705,tt8290698,The Spy Gone North,Drama,Jong-bin Yoon,"[18, 36]",517991,ko,공작,6.794,2018-08-17,7.3,33,CJ,501000.0,1200000.0,2018
3706,tt8404272,How Long Will I Love U,Romance,Lun Su,"[35, 10749, 18, 9648]",522921,zh,超时空同居,6.840,2018-05-17,7.4,11,WGUSA,747000.0,82100000.0,2018
3707,tt8851262,Spring Fever,"Comedy,Horror","Chris McElyea,Izzy Sutton","[18, 10749]",117646,zh,春风沉醉的夜晚,2.277,2010-08-06,5.9,8,Strand,10800.0,150000.0,2010


In [72]:
merged_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3709 entries, 0 to 3708
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           3709 non-null   object 
 1   title              3709 non-null   object 
 2   genre              3709 non-null   object 
 3   director           3709 non-null   object 
 4   genre_ids          3709 non-null   object 
 5   id                 3709 non-null   int64  
 6   original_language  3709 non-null   object 
 7   original_title     3709 non-null   object 
 8   popularity         3709 non-null   float64
 9   release_date       3709 non-null   object 
 10  vote_average       3709 non-null   float64
 11  vote_count         3709 non-null   int64  
 12  studio             3709 non-null   object 
 13  domestic_gross     3709 non-null   float64
 14  foreign_gross      3709 non-null   float64
 15  year               3709 non-null   int64  
dtypes: float64(4), int64(3),